# 1. Initalize Script

If you're running this script on Google Colab<br>
Mount your Google drive: 
1. Click on the folder icon on the left
2. Click Mount Drive
3. The root directory would be /content/
```
# your Google Drive folder would be at:
/content/drive/My Drive/
```

Change working directory:<br>
1. Run this command:
```
%cd /content/drive/My Drive/<your folder>
```

In [1]:
%cd /content/drive/My Drive/Data Science/Stock Prediction

/content/drive/My Drive/Data Science/Stock Prediction


# Tweets

In [7]:
from dotenv import load_dotenv
load_dotenv(verbose=True)
import os
import tweepy

# Creating the authentication object
auth = tweepy.OAuthHandler(os.getenv('consumer_key'), os.getenv('consumer_secret'))
# Setting your access token and secret
auth.set_access_token(os.getenv('access_token'), os.getenv('access_token_secret'))

# Creating the API object while passing in auth information
tweet_api = tweepy.API(auth)

In [8]:
from src.helper import pickle_load
folder = './data'
filename = 'tweets_df'
tweets = pickle_load(f'{folder}/{filename}.pkl').set_index('id').T.to_dict()

In [ ]:
import time
def limit_handled(cursor):
    while True:
        try:
            yield cursor.next()
        except StopIteration:
            print('StopIteration, Wait for 15mins...')
            time.sleep(15 * 60)
        except tweepy.RateLimitError:
            print('RateLimitError, Wait for 15mins...')
            time.sleep(15 * 60)

from src.data_mining import organize_tweet
for tweet in limit_handled(tweepy.Cursor(tweet_api.user_timeline, screen_name='BusinessTimes').items()):
    tweet_json = tweet._json
    if tweet_json.get('id_str') in tweets: continue

    tweet = organize_tweet(tweet_json)
    tweet_id = tweet.pop('id')
    tweets.update(
        {
            tweet_id: tweet
        }
    )

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
StopIteration, Wait for 15mins...


In [ ]:
len(tweets)

In [ ]:
from src.helper import pickle_dump
folder = './data'
filename = 'tweets_df'
pickle_dump(f'{folder}/{filename}.pkl', pd.DataFrame.from_dict(tweets).T.reset_index().rename(columns={'index':'id'}))